In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("C:/Users/Ishit/Desktop/train.csv")
final_test=pd.read_csv("C:/Users/Ishit/Desktop/test.csv")
data=data.loc[:,data.columns != "id"]
final_test=final_test.loc[:,final_test.columns != "id"]
data=data.replace({"Class_1"},0)
data=data.replace({"Class_2"},1)
data=data.replace({"Class_3"},2)
data=data.replace({"Class_4"},3)
data=data.replace({"Class_5"},4)
data=data.replace({"Class_6"},5)
data=data.replace({"Class_7"},6)
data=data.replace({"Class_8"},7)
data=data.replace({"Class_9"},8)

col = [str(i) for i in range(0,75)]
final_test.columns = col
col.append('target')
data.columns = col
col.pop(-1)
data[col] = data[col]/data[col].max()

split=160000
x = data[:split][col]
y = data[:split].target
xt = data[split:][col]
yt = data[split:].target
x=np.array(x)
y=np.array(y)
xt=np.array(xt)
yt=np.array(yt)
print(x.shape, xt.shape)

(160000, 75) (40000, 75)


In [21]:
def sigmoid(x):
    return(np.divide(1,(1+np.exp(-x))))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

def output_formula(x, weights, bias):
    return(sigmoid(np.dot(x,weights)+bias))

def error_formula(y, output):
    return(-(np.dot(y.T,np.log(output))+np.dot((1-y).T,np.log(1-output))))

def error_term_formula(x, y, output):
    return (y-output)*np.dot(sigmoid_prime(x),x.T)

def update_weights(x, y, weights, bias, learningrate):
    weights=weights + learningrate*((y-output_formula(x,weights,bias))*x)
    bias=bias+learningrate*(y-output_formula(x,weights,bias))
    return weights,bias

In [24]:
np.random.seed(44)
epochs=100
learningrate=0.0001

def train(features, target, epochs, learningrate):
    ans=[]
    for i in range(0,9):
        ptarget=(target==i).astype(int)
        n,m=features.shape
        weights = np.random.normal(scale=1 / m**.5, size=m)
        bias=0
        for t in range(epochs):
            correction = np.zeros(weights.shape)
            for x,y in zip(features,ptarget):
                tp = sigmoid(np.dot(x,weights))
                error = error_term_formula(x,y,tp)
                correction = correction+error
            weights = weights+learningrate*correction
            calculated_target = sigmoid(np.dot(features,weights))
            cost = np.mean(error_formula(ptarget,calculated_target))
            if t%(5)==0:
                print("\n   Epoch:",t,"   Cost:",cost)
        ans.append(weights)
    return ans;

In [25]:
ans=train(x,y,epochs,learningrate)


   Epoch: 0    Cost: 62904.79001703503

   Epoch: 5    Cost: 50717.05534686635

   Epoch: 10    Cost: 47522.44243325926

   Epoch: 15    Cost: 47308.59226349238

   Epoch: 20    Cost: 47298.82662657573

   Epoch: 25    Cost: 47298.00690377664

   Epoch: 30    Cost: 47297.92188027403

   Epoch: 35    Cost: 47297.9128108896

   Epoch: 40    Cost: 47297.911840485714

   Epoch: 45    Cost: 47297.911736620394

   Epoch: 50    Cost: 47297.91172550297

   Epoch: 55    Cost: 47297.91172431299

   Epoch: 60    Cost: 47297.911724185615

   Epoch: 65    Cost: 47297.91172417198

   Epoch: 70    Cost: 47297.911724170524

   Epoch: 75    Cost: 47297.911724170364

   Epoch: 80    Cost: 47297.91172417035

   Epoch: 85    Cost: 47297.91172417035

   Epoch: 90    Cost: 47297.91172417034

   Epoch: 95    Cost: 47297.91172417034

   Epoch: 0    Cost: 92351.50973540916

   Epoch: 5    Cost: 61223.425204499246

   Epoch: 10    Cost: 60360.931772869146

   Epoch: 15    Cost: 60360.52251163802

   Epoch: 20 

In [46]:
mytest=pd.read_csv("C:/Users/Ishit/Desktop/test.csv")
yy=mytest["id"]
yy=np.array(yy)
yy.resize(100000,1)
print(yy.shape)
print(mytest.shape)
final_test=final_test.astype(float)
final_test=np.array(final_test)

(100000, 1)
(100000, 76)


In [81]:
tpp=yy
tpp=np.array(tpp)
tpp.resize(100000,9)
tpp=tpp.astype(float)
j=0;
for x in final_test:
    final_ans=[]
    for i in range(0,9):
        weights=ans[i]
        output=output_formula(x,weights,0)
        final_ans.append(output)
    tpp[j]=final_ans
    j=j+1
        

<ipython-input-21-44cd7f3ac619>:2: RuntimeWarning: overflow encountered in exp
  return(np.divide(1,(1+np.exp(-x))))


In [104]:
myans=pd.DataFrame({"id":yy[:,0]})
for i in range(0,9):
    temp = pd.DataFrame({"Class_{}".format(i): tpp[:, i]})
    myans = pd.concat([myans,temp], axis = 1)
for i in range(0,9):
    myans["Class_{}".format(i)]=myans["Class_{}".format(i)].round(decimals=4)

In [103]:
myans.to_csv('submission.csv',index=False,header=True)